# Welcome to the Advanced Interactive User Guide for KALMUS (API)!

In this notebook, I will introduce:

1. **The major components of a Barcode visualization**:
    - what are the concepts that you could extend or change to incorporate your own ideas
    - what are the tools you could readily use in creating your own barcodes
    
2. **How to create your own Barcode Class by inheriting either:**
    - ColorBarcode
    - BrightnessBarcode
    
3. **Some real examples that overrides:**
    - `process_frame` method of Barcode Class
    - `get_color_from_frame` method of Barcode Class  
    - or both  
    to create new Barcode Class that retrieve information from film that is even outside Color and Brightness!

# 0. Import kalmus modules

In [ ]:
from kalmus.barcodes.Barcode import ColorBarcode, BrightnessBarcode
from kalmus.barcodes.BarcodeGenerator import BarcodeGenerator
import kalmus.utils.artist as artist
import matplotlib.pyplot as plt
import numpy as np

# 1. What are the important components of a Barcode?

**Barcode Class** ([API References](https://kalmus-color-toolkit.github.io/KALMUS/barcodes/BarcodeBase.html)):
- **attributes**:  
    1. **color_metric(str)**: The measure of color (or brightness for brightness barcode)  
    2. **frame_type(str):** The type of extracted region/region of interest (e.g. Foreground or Region with High [brightness contrast](https://www.w3.org/TR/2008/REC-WCAG20-20081211/#contrast-ratiodef))
    3. **barcode_type(str):** The type of barcode visualization (`"Color"` for `ColorBarcode` or `"Brightness"` for `BrightnessBarcode`).
    4. **skip_over(int):** How many frames to skip at the start before collecting color/brightness
    5. **sampled_frame_rate(int):** The period of one frame being collected for measuring color.
    6. **total_frames(int):** Total number of frames being included in the barcode
    7. **video(class: `cv2.VideoCapture`):** The input video read as a `cv2.VideoCapture` object
    8. **film_length_in_frames(int):** The total number of frames in the input video.
    9. **fps(float):** The frame per second rate of input video (how many frames are there in one second of input video).
    10. **meta_data(dict):** A python dictonary object that records the meta information of the barcode.
- **methods**:
    1. **read_videos(str video_path_name):** Read in the video at video_path_name as a cv2.VideoCapture object. By default, also find the letterbox position. If save_frames_in_generation during the generation, it also determines the appriorate saved_frames_sampled_rate from the sampled_rate given by the users.
    2. **find_film_letterbox()**: Automatically find the black/dark letterbox around the film frame.
    3. **remove_letter_box_from_frame(numpy.ndarray frame)**: Remove the letterbox from the input frame.
    4. **process_frame(numpy.ndarray frame)**: Remove the letterbox of frame using `self.remove_letter_box_from_frame()` method and then extract the region of interest following `self.frame_type`.
    5. **get_color_from_frame(numpy.ndarray frame)**: Measure the color/brightness (notice that this method also collect brightness, brightness can be seen a "color" with only just one channel value) in input frame.
    6. **set_letterbox_bound(int up_vertical_bound, int down_vertical_bound, int left_horizontal_bound, int right_horizontal_bound)**: Tool method for setting up the letterbox position manually and tells barcode use manual setting during generation.
    7. **enable_save_frames(float sampled_rate)**: Enable barcode to save frame during the generation for later visualization. Save one frame every sampled_rate seconds.
    8. **enable_rescale_frames_in_generation(float rescale_factor)**: Enable barcode to rescale frame before processing frame or collecting color/brightness.
    9. **save_as_json(str filename)**: Method to save the current barcode object into a JSON object that can be later loaded and rebuilt using `BarcodeGenerator.generate_barcode_from_json` method.
    10. **get_barcode()**: Get method that returns the barcode image (plottable visualization).

---
**ColorBarcode Class inherited Barcode Class** ([API References](https://kalmus-color-toolkit.github.io/KALMUS/barcodes/ColorBarcode.html)):
- **attributes**:
    1. **Barcode attributes above**
    2. **colors(numpy.ndarray)**: A numpy.ndarray with array of three-channel RGB colors collect from frames during generation. shape==(`self.total_frames`, 3)
    
- **methods**:
    1. **collect_colors(str video_path_name)**: Generate the color barcode using current setting from input video at video_path_name. (Single thread)
    2. **multi_thread_collect_colors(str video_path_name, int num_thread)**: Similar to `collect_colors` but with multi-threading.
    3. **reshape_barcode(int frames_per_column)**: Reshape the `self.colors` into a 2D barcode image with shape==(frames_per_column, total_frames // frames_per_column, 3).
    
---
**BrightnessBarcode Class inherited Barcode Class** ([API References](https://kalmus-color-toolkit.github.io/KALMUS/barcodes/BrightnessBarcode.html)):
- **attributes**:
    1. **Barcode attributes above**
    2. **brightness(numpy.ndarray)**: A numpy.ndarray with array of one-channel RGB colors collect from frames during generation. shape==(`self.total_frames`, 1)
    
- **methods**:
    1. **collect_brightness(str video_path_name)**: Generate the brightness barcode using current setting from input video at video_path_name. (Single thread)
    2. **multi_thread_collect_brightness(str video_path_name, int num_thread)**: Similar to `collect_brightness` but with multi-threading.
    3. **reshape_barcode(int frames_per_column)**: Reshape the `self.colors` into a 2D barcode image with shape==(frames_per_column, total_frames // frames_per_column).
    
---

## 1.1 Umm... That's a lot.

But notice that **most of them are just tools** rather than conceptual components that control what information is included in the barcode, or how the final barcode visualization will look like. For example, it is likely you always wish to get rid of the letterboxing around the film frames since there is no meaningful information in them. `find_film_letterbox` and `remove_letter_box_from_frame` will be the handy tools for you to take off annoyed letterboxing.

However, the `process_frame` and `get_color_from_frame` are more related to the conceptual part of a barcode visualizaton:
- What type of region in each frame should be extracted for measuing colors?
- What kind of measures should I apply to get the color (Average, mode, dominant, or some of my own ideas)?

In the next part of this notebook, I will show you how to override the ColorBarcode/BrightnessBarcode class to **embrace your own designs** yet with cost of only a few efforts by taking advantage of  **existing tools**. 

# 2. How about baking a Barcode with your own recipe of frame and color/brightness?

Remember that in [our previous guide](user_guide_for_kalmus_api.ipynb) for kalmus's API, we talked about a scenario that you (or anyone) wish to use the **original frame with black lettebox** when generating average color ColorBarcode.

Previously, we achieve this by define our own letterbox position in the `BarcodeGenerator.generate_barcode` method, which is also equivalent to the `Barcode.set_letterbox_bound` method:
```python
# Suppose barcode is a ColorBarcode or BrightnessBarcode object
# Let user to set letterbox position
# The film frame's shape is 720 x 1280 x 3 == height x width x channels
barcode.set_letterbox_bound(up_vertical_bound=0, down_vertical_bound=720, left_horizontal_bound=0, right_horizontal_bound=1280)
```

**This relies on our knowledge about the input video**: we need to know the original shape/spatial size of input video's frame.
**We cannot generalize this to the other input video**: different input video has different shape/spatial size for their frames. If we wish to also use the original frame when processing the other videos, we need to change the parameters of letterbox position accordingly. This brings in a lot of work!

**There is a much better way for you to achieve this by providing Barcode class with you own recipe of frame**: what kind of frame you wish to use in your barcode generation.

## 2.1 Creating your own Barcode class with customized process_frame method

`Barcode.process_frame` method process the raw frame following the specified frame_type. Its basical workflow:
- Remove the letterboxing of the input raw frame based on the automatically found/user defined letterbox position
- Process the frame and extract the regions of interest (for example, foreground part of a frame) based on the specified frame_type
- return the processed frame for measuring color in next step

Above are the abstraction of a typical workflow for `Barcode.process_frame`. However, since we will override the `Barcode.process_frame` method with our new recipe, we would like some implementation details of **process_frame** method.  

1. First, the **process_frame** is class method of `Barcode` class and is inherited in both `ColorBarcode` and `BrightnessBarcode`.
2. Second, the input of the **process_frame** method is the original frame in **RGB colorspace** read from input video. The image is a `numpy.ndarray` with **shape==(height, width, channels (3))** and `dtype=="uint8"`. 
3. **Important**: notice that no matter it is in the `ColorBarcode` or `BrightnessBarcode`, the input to the **process_frame** method is alway an **color image** in **RGB colorspace**. The conversion of color image to grayscale image (RGB colorspace to grayscale) happens after the **process_frame** method but before the **get_color_from_frame** method (which I will cover later).
4. Finally, the output of the **process_frame** method is the processed frame or number of pixels (flattened image) in **RGB colorspace**, which is still a `numpy.ndarray` with `dtype=="uint8"` but its shape can be either **==(height, width, channels(3))** for image-like output (for example, whole frame without letterboxing), or **==(number of pixels, channels(3))** for flattened-image output (for example, the first row of pixels in input frame).

The **only two things** that your overrided `process_frame` method needed to follow are: 
1. **Has valid operations on the input color frame, which is in type `numpy.ndarray` with `dtype=="uint8"`**
2. **Output is a `numpy.ndarray` that has the expected shape with `dtype=="uint8"`.**

## 2.2 Let's start baking...

Our first customized barcode will use the ColorBarcode as the baking mold (or, simply, as the parent class), and we will override the `ColorBarcode.process_frame` to let the barcode use the whole frame with letterboxing in generation (**no more `ColorBarcode.remove_letter_box_from_frame`**).

In [ ]:
class MyBarcode(ColorBarcode):
    def process_frame(self, frame):
        return frame

## 2.3 Well..... Yep, that's it!

To use the original frame with everything (including letterbox), just **override the `process_frame` method with `return frame`**. Now, your barcode will use the original frame in generation.

Let's take a try on our [example data](notebook_example_data/i_robot_video.mp4). **Again,** feel free to replace it with any videos/films that are available to you!

In [ ]:
# Feel frame to tell the constructor that frame_type is "" or any name you like, you have overrided the process_frame with your own recipe.
my_barcode = MyBarcode("Average", frame_type="my_recipe", skip_over=10, total_frames=180, sampled_frame_rate=1)
# The multi_thread_collect_colors is like the generate_barcode method of BarcodeGenerator but much simple
# Only two parameters need to be specified: (1) path to video file, (2) number of threads used in generation
# Feel free to adjust them
my_barcode.multi_thread_collect_colors("notebook_example_data/i_robot_video.mp4", num_thread=4)
my_barcode.reshape_barcode(10)

## 2.4 Let's take a quick look on what we have

In [ ]:
plt.figure()
plt.imshow(my_barcode.get_barcode())
plt.show()

Looks right, but maybe let us also generate the barcode with original design that **removes all letterboxing** so we can have a comparison.

---

## 2.5 Generate the barcode with original ColorBarcode

We still use Average color but **whole frame without letterboxing**.

In [ ]:
color_barcode = ColorBarcode("Average", frame_type="Whole_frame", skip_over=10, total_frames=180, sampled_frame_rate=1)
color_barcode.multi_thread_collect_colors("notebook_example_data/i_robot_video.mp4", num_thread=4)
color_barcode.reshape_barcode(10)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
ax[0].set_title("Remove letterboxing")
ax[0].imshow(color_barcode.get_barcode())

ax[1].set_title("Keep letterboxing")
ax[1].imshow(my_barcode.get_barcode())

plt.tight_layout()
plt.show()

**Ha!** We made it using only three lines of code. We create a new ColorBarcode class that always use the whole frame with everything in barcode generation. This is a generalizable solution that can applied to any other videos without knowing any meta information of those videos.

---

## 2.6 However, we can make it even more exciting!

Notice that in the above example, we only **override the `process_frame` method**, but there is another concept which is readily extendable with new implementation (your recipes). 

`Barcode.get_color_from_frame` method workflow:
- Compute the color/brightness of the input color/grayscale frame
- Return the three-channel RGB color or single-channel brightness

That's it!

Very simple abstract workflow, and its [implementation details](https://kalmus-color-toolkit.github.io/KALMUS/_modules/kalmus/barcodes/Barcode.html#Barcode.get_color_from_frame) is also an easy one. 
1. First the input frame is the processed frame from `process_frame` method (with colorspace being converted to grayscale for BrightnessBarcode). It is alway a 2D image (color image for ColorBarcode, grayscale image for BrightnessBarcode), which is a `numpy.ndarray` with 3 (color) or 2 (grayscale) dimensions and `dtype=="uint8"`. For ColorBarcode shape==(height, width, channels(3)), or for BrightnessBarcode shape==(height, width).
2. **Notice that** if your overrided `process_frame` method returns a flatten image, shape==(number of pixels, channels(3)), the input frame will still be 2D image with shape==(number of pixels, 1, channels(3)) or shape==(number of pixels, 1).
3. The return color must be a `numpy.ndarray` with shape==(3,) and `dtype=="uint8"`. The return brightness must be a `numpy.ndarray` with shape==(1,) and `dtype=="uint8"`.

**Again**, the only two things your overrided `get_color_from_frame` need to follow are:
1. **Has valid operations on the input numpy.ndarray frame (RGB color frame for ColorBarcode or grayscale frame for BrightnessBarcode).**
2. **The out is a numpy.ndarray with expected shape and `dtype=="uint8"`**

In fact, **everything** inside the **ColorBarcode/BrightnessBarcode** class can be easily overrided to incorporate with your own designs. Not only the **conceptual parts** (like `process_frame` for extracing regions of interest or `get_color_from_frame` for measuring color), but you could also override the **tools**, such as `find_film_letterbox`, to fit your special needs. For example, currently `find_film_letterbox` only works for dark/black letterboxing, but you could override it to detect white/colorful letterboxing in your special video data.

## 2.7 Okay, let's see what are in our new recipe!

So, this time we will use the **only central one ninth part of the frame** (if you divide the frame into 9 equal width and equal height sector, you only use the central one in barcode generation). We believe the character's faces are mainly in that one ninth center.

Then, we will also customize our color recipe. Remember that there is a color_metric==**Top-dominant**. We will use the same idea but somehow opposite way! We will collect the **Least dominant** color from the **one ninth center of the frame.**

Great! Let's do it!

In [ ]:
class MyBarcodeLeastDominant(ColorBarcode):
    def __init__(self, color_metric="", frame_type="", sampled_frame_rate=1, skip_over=10, total_frames=180, barcode_type="Color"):
        super().__init__(color_metric, frame_type, sampled_frame_rate, skip_over, total_frames, barcode_type)
    
    
    def process_frame(self, frame):
        # This will allow you to use the self.enable_rescale_frames_in_generation() to speed up generation by downsizing the input frame
        if self.rescale_frames_in_generation:
            frame = self._resize_frame(frame)
        return frame[frame.shape[0] // 3: frame.shape[0]* 2 // 3, frame.shape[1] // 3: frame.shape[1] * 2 // 3]
    
    
    def get_color_from_frame(self, frame):
        colors, dominances = artist.compute_dominant_color(frame, n_clusters=3)
        pos = np.argsort(dominances)[0]
        color = colors[pos]
        
        return color

## 2.8 Generate it!

In [ ]:
barcode = MyBarcodeLeastDominant(sampled_frame_rate=1, skip_over=10, total_frames=180)
# Remember these are just the handy tools available for you to use
# We want to save the frame for later visualization
barcode.enable_save_frames(sampled_rate=1)
# And the current video in notebook_example_data has a 1K resolution. 
# Let's rescale a little bit to get a faster generation speed
# rescale_factor==0.16 => rescale_width = 0.4 * original_width and rescale_height = 0.4 * original_height
barcode.enable_rescale_frames_in_generation(rescale_factor=0.16)
barcode.multi_thread_collect_colors("notebook_example_data/i_robot_video.mp4", num_thread=4)
barcode.reshape_barcode(10)

## 2.9 Let's see what do we have!

In [ ]:
plt.figure()
plt.imshow(barcode.get_barcode())
plt.show()

**Looks so much different, doesn't it?**

---

## 2.10 Let's check the saved frame to see if this make sense!

In [ ]:
# Just import draw utility for me to mark that one ninth center in saved frames
from skimage.draw import rectangle_perimeter

fig, ax = plt.subplots(1, barcode.saved_frames.shape[0] // 2, figsize=(16, 5))
for i in range(len(ax)):
    image = barcode.saved_frames[i * 2]
    rr, cc = rectangle_perimeter(start=(image.shape[0] // 3, image.shape[1] // 3), 
                                 end=(image.shape[0] * 2 // 3, image.shape[1] * 2 // 3))
    
    image[rr, cc] = [255, 0, 0]
    ax[i].imshow(image)
    ax[i].axis("off")
plt.tight_layout()
plt.show()

## 2.11 Make it even cooler!

Let's try to tackle a real-world problem!

There are so many **eye tracking** studies, and many of them record the participants' **eye fixation on a screen** frame when they are watching a video or movie! Are you curious about what **kind of color is in the region around that fixation**? I do! Let's take advantage of a large published eye tracking datasets that not only has the available eye tracking data but also the original movie clips. We will figure what color did the participants saw on the screen when they are watching those movie clips. 

Go to the ETMD eye tracking project website: [http://cognimuse.cs.ntua.gr/etmd](http://cognimuse.cs.ntua.gr/etmd)

On the ETMD dataset website, scroll down and find a link of **Download ETMD Video Clips**.
![ETMD](notebook_figures/ETMD_dataset.png)

1. Click the link and download the original movie clip, 
2. Unzip the downloaded file 
3. Find a movie clip called **CHI_1_color.avi**
4. Put it into the notebook_example_data folder
5. That's it

## 2.12 Read in the Fixtaion dataframe

The dataframe has three columns:
1. Fixation X: the relative position of fixation in x axis (corresponding to columns of numpy.ndarray image). Range [0, 1]
2. Fixation Y: the relative position of fixation in y axis (corresponding to rows of numpy.ndarray image). Range [0, 1]
3. Valid Fixation: If the fixation is a valid fixation or not.

Each observation is corresponding to one frame of the input video.

In [ ]:
import pandas as pd

data = pd.read_csv("notebook_example_data/CHI_1_eye_track_data.csv")
data

## 2.12 Let's create our FixationBarcode

The fixation barcode will extract the region around the fixation point and measure the color on that region.

Therefore, we need to **override the process_frame** method so it returns the extracted **fixation region**.

Let me defined the **fixation region** as a square region with fixation as the center and **expand 25 pixels** to up, down, left, and right.

In [ ]:
class FixationBarcode(ColorBarcode):
    def __init__(self, color_metric="Average", frame_type="", sampled_frame_rate=1, skip_over=10, total_frames=180, barcode_type="Color",
                 fixation_df=None, extent=25):
        super().__init__(color_metric, frame_type, sampled_frame_rate, skip_over, total_frames, barcode_type)
        # pandas.DataFrame object
        self.fixation_df = fixation_df
        # Size of the fixation region
        self.extent = extent
        # Index record the index of current processing frame
        self.cur_frame_index = skip_over
        
    
    
    def process_frame(self, frame):
        valid = self.fixation_df["Valid Fixation"][self.cur_frame_index]
        
        # If it is not a valid fixation return a black frame
        if valid != 1:
            extracted_region = np.zeros(shape=(10, 10, 3))
            self.cur_frame_index += self.sampled_frame_rate
            return extracted_region
        
        # Get relative position of fixation on x and y axis
        fixtaion_x = self.fixation_df["Fixation X"][self.cur_frame_index]
        fixtaion_y = self.fixation_df["Fixation Y"][self.cur_frame_index]
        
        # Convert them into the y => row index and x => col index
        index_row = int(frame.shape[0] * fixtaion_y)
        index_col = int(frame.shape[1] * fixtaion_x)
        
        # Get the coordinates with the size of extent
        coordinates = np.array([[index_row - self.extent, index_col - self.extent],
                                [index_row + self.extent, index_col + self.extent]])
        
        # Clip the coordinates to avoid out of range
        coordinates[:,0] = coordinates[:,0].clip(0, frame.shape[0])
        coordinates[:,1] = coordinates[:,1].clip(0, frame.shape[1])
        
        extracted_region = frame[coordinates[0, 0]: coordinates[1, 0], coordinates[0, 1]: coordinates[1, 1]]
        # If the fixation is invalid => extracted region is too small
        if extracted_region.size < 30:
            extracted_region = np.zeros(shape=(10, 10, 3))
        
        self.cur_frame_index += self.sampled_frame_rate
        
        return extracted_region

## 2.13 Important, you have to use the single-threaded collect_colors in this case

Since we rely on a new class attribute **self.cur_frame_index** to get the corresponding fixation data in the dataframe, we have to use the single threaded barcode generate method `collect_colors`.

This class attribute, though, is shared within class but not **synchronized** between the threads.

In [ ]:
# Feel frame to change the color_metric with the other available options to visualize the difference
color_metric = "Median"
fixation_barcode = FixationBarcode(color_metric=color_metric, fixation_df=data, sampled_frame_rate=1, skip_over=0, total_frames=1e8)

fixation_barcode.enable_save_frames(sampled_rate=4)
fixation_barcode.collect_colors("notebook_example_data/CHI_1_color.avi")
fixation_barcode.reshape_barcode(50)

In [ ]:
# Also compute the original barcode for comparison
# Using the same color metric
original_barcode = ColorBarcode(color_metric=color_metric, frame_type="Whole_frame", sampled_frame_rate=1, skip_over=0, total_frames=1e8)

original_barcode.enable_save_frames(sampled_rate=4)
original_barcode.collect_colors("notebook_example_data/CHI_1_color.avi")
original_barcode.reshape_barcode(50)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].set_title("Fixation Barcode")
ax[0].imshow(fixation_barcode.get_barcode())

ax[1].set_title("Original Whole_frame Barcode")
ax[1].imshow(original_barcode.get_barcode())

plt.tight_layout()
plt.show()

We, as humans, have the ability to find and cosume **the more interesting information** available to us. Look at the **fixation barcode**, I hope you are amazed how much brighter and vivider those colors are compared with the unpolished raw frame.

---

## Let's take a quick look on the saved frame with marked fixation region (in red box)

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(16, 5))
for i in range(len(ax)):
    image = fixation_barcode.saved_frames[i * 7 + 3].copy()
    
    fixtaion_x = data["Fixation X"][fixation_barcode.saved_frames_sampled_rate * (i * 7 + 3)]
    fixtaion_y = data["Fixation Y"][fixation_barcode.saved_frames_sampled_rate * (i * 7 + 3)]

    index_row = int(image.shape[0] * fixtaion_y)
    index_col = int(image.shape[1] * fixtaion_x)
    
    rr, cc = rectangle_perimeter(start=(index_row - 15, index_col - 15), extent=(15, 15))
    
    image[rr, cc] = [255, 0, 0]
    ax[i].imshow(image)
    ax[i].axis("off")
plt.tight_layout()
plt.show()

# 3. Your barcode could do more than color and brightness!

**Remember!** Barcode is just a concept of transforming the 3D (spatial-temporal) information of media data into a 2D (spatial) visualization that fits the need of a direct while comprehensive view of media data.

There are certainly **many frame-level information** other than color/brightness embedded in the media data. The abstraction of Barcode allows you to **make design that embrace your own concept** based on the provided infrastructures (ColorBarcode/BrightnessBarcode).

**Notice that** ColorBarcode and BrightnessBarcode only define (not constrain!) what colorspaces of frames (of input video) will be used when generating barcode, but not the colorspace of your barcode visualization. In the following examples, I will show you how to create a Brightness/grayscale visualization using ColorBarcode, and how to create a heatmap visualization using BrightnessBarcode.

## 3.1 Let's first work on a ColorBarcode with Heatmap visualization

Inside our kalmus.utils.artist module, there is a function called `artist.watershed_segmentation` that could segment an input frame into several regions based on the idea of edge detection (check [here](https://www.sciencedirect.com/topics/computer-science/watershed-segmentation) to what is the watershed segmentation or here to see its [API references](https://kalmus-color-toolkit.github.io/KALMUS/utils/artist.html#kalmus.utils.artist.watershed_segmentation)).

Let me hypothesize that the number of regions segmented by **watershed segmentation** in a frame has a correlation with complexity of that frame's content: the larger the number of regions, the more complex that frame is. Furthermore, the more complex a frame, it is more likely the main action is happening in that frame.

Very cool idea! Let's see how could we create a **Heatmap barcode** that would help us to visualize this hypothesized relations. So, the basic idea is:
- override the `process_frame` method so it returns a frame without letterboxing
- override the `get_color_from_frame` method so it segments the frame using `artist.watershed_segmentation` and get the number of regions
- remember that `get_color_from_frame` method need to return a **three-channel RGB color** value at the end. To follow this, we will use the **color map of matplotlib module**, which will help us to convert the number of regions into a heatmap color that represents the intensity (or complexity of frame in this case).

In [ ]:
import matplotlib as mpl
import numpy as np


class ComplexityBarcode(ColorBarcode):
    def __init__(self, color_metric="", frame_type="", 
                 sampled_frame_rate=1, skip_over=10, total_frames=180, 
                 barcode_type="Color", cmap="bwr", min_val=1, max_val=20):
        super().__init__(color_metric, frame_type, sampled_frame_rate, skip_over, total_frames, barcode_type)
        self.colormap_func = mpl.cm.get_cmap(cmap)
        self.min_val = min_val
        self.max_val = max_val
    
    def process_frame(self, frame):
        # This will allow you to use the self.enable_rescale_frames_in_generation() to speed up generation by downsizing the input frame
        if self.rescale_frames_in_generation:
            frame = self._resize_frame(frame)
        # Return a frame without letterboxing
        return self.remove_letter_box_from_frame(frame)
    
    
    def normalize_data(self, data, min=1, max=20):
        return (data - min) / (max - min)
    
    
    def get_color_from_frame(self, frame):
        labels, _ = artist.watershed_segmentation(frame)
        
        num_regions = np.unique(labels).size
        norm_data = self.normalize_data(num_regions, self.min_val, self.max_val)

        color = self.colormap_func(norm_data)[:3] 
        color = np.array(color) * 255
        
        return color.astype("uint8")

## 3.2 Let me say a typical frame with 20 regions is complex enough, while a frame always have at least one region

Therefore, we define the min_val for data normalization to be 1 and max_val for data normalization to be 20.

In [ ]:
com_barcode = ComplexityBarcode(sampled_frame_rate=1, skip_over=0, total_frames=1e8, 
                                min_val=1, max_val=20, cmap="Reds")

com_barcode.enable_save_frames(sampled_rate=0.1)
com_barcode.enable_rescale_frames_in_generation(rescale_factor=0.16)
com_barcode.multi_thread_collect_colors("notebook_example_data/i_robot_video.mp4", num_thread=6)
com_barcode.reshape_barcode(15)

## 3.3 How does it look like?

In [ ]:
plt.figure()
plt.imshow(com_barcode.get_barcode())
plt.show()

Well, it looks like in some frame there indeed many regions (a complex frame). However, we could check in this with more details. Notice that we have **saved frames** during the generation. **Save this barcode into a JSON file and load it back to GUI, and then we can check the frames at high complexity pixel by doubling clicking that point on the plot.** How to do that? Check our user guide for [KALMUS's GUI](user_guide_for_kalmus_gui.ipynb).

## 3.4 Save our Complexity Barcode 

You could save your customized barcode in the way that you save ColorBarcode or BrightnessBarcode.

**One notes:** You have to **drop your color map function** before saving since it is not serializable.

In [ ]:
import os

com_barcode.colormap_func = None
com_barcode.save_as_json("Complexity_barcode.json")
os.path.exists("Complexity_barcode.json")

## 3.5 Create a Heatmap Barcode based on BrightnessBarcode

There is actually a more formal approach to measure the complexity (or uncertainty) of the information -- **[Information entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory))**.

In the next example, I will show how to use the `shannon_entropy` function of skimage.measure module to create a heatmap barcode visualization that show the entropy of film frames along the time.

`shannon_entropy` only accepts the grayscale image as the input. The BrightnessBarcode becomes the perfect choice for this one.

## 3.6 The infrascture is almost the same

In [ ]:
from skimage.measure import shannon_entropy


class EntropyBarcode(BrightnessBarcode):
    def __init__(self, color_metric="", frame_type="", 
                 sampled_frame_rate=1, skip_over=10, total_frames=180, 
                 barcode_type="Brightness", min_val=0, max_val=8):
        super().__init__(color_metric, frame_type, sampled_frame_rate, skip_over, total_frames, barcode_type)
        self.min_val = min_val
        self.max_val = max_val
    
    def process_frame(self, frame):
        # This will allow you to use the self.enable_rescale_frames_in_generation() to speed up generation by downsizing the input frame
        if self.rescale_frames_in_generation:
            frame = self._resize_frame(frame)
        # Return a frame without letterboxing
        return self.remove_letter_box_from_frame(frame)
    
    
    def normalize_data(self, data, min=0, max=8):
        return (data - min) / (max - min)
    
    
    def get_color_from_frame(self, frame):
        entropy = shannon_entropy(frame)
        norm_entropy = self.normalize_data(entropy, self.min_val, self.max_val) * 255
        
        return np.array(norm_entropy, dtype="uint8").reshape(1,)

## 3.7 Generate it!

In [ ]:
entropy_barcode = EntropyBarcode(sampled_frame_rate=1, skip_over=0, total_frames=1e8, 
                                 min_val=0, max_val=8)

entropy_barcode.enable_save_frames(sampled_rate=0.1)
entropy_barcode.enable_rescale_frames_in_generation(rescale_factor=0.16)
entropy_barcode.multi_thread_collect_brightness("notebook_example_data/i_robot_video.mp4", num_thread=6)
entropy_barcode.reshape_barcode(15)

## 3.8 To view as a Heatmap rather than a grayscale brightness image, just change the cmap

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(14, 3))

for i, cmap in enumerate(["gray", "Reds", "YlGn", "coolwarm"]):
    ax[i].imshow(entropy_barcode.get_barcode(), cmap=cmap)

plt.tight_layout()
plt.show()

## 3.9 And that's it!

# 4. Thank you

**and Congraluations! You have just finished the Advanced tutorial on KALMUS API.**

If you have any problems in running this notebook, please check the README.md file in this folder for trouble shooting. If you find any errors in the instructions, please feel free to email the notebook author, Yida Chen, <yc015@bucknell.edu>